## xcube Data Store Framework - GeoTIFF and Cloud Optimized GeoTIFF (COG)

*Please checkout the general introduction to xcube data stores in the Jupyter Notebook [Getting Started](./1_getting_started.ipynb) before jumping into this notebook :)* 

This notebook explains how to open multi-resolution datasets (image pyramids) from a COG file using the `file` or `s3` data stores.

In [ ]:
from xcube.core.new import new_cube
from xcube.core.store import find_data_store_extensions
from xcube.core.store import get_data_store_params_schema
from xcube.core.store import new_data_store
from xcube.core.store import new_data_writer

### Getting prepared

Get a data store instance. We use the local filesystem here ("file"), you can also AWS S3 ("s3") if you have a writable bucket (= root).

In [ ]:
data_store = new_data_store("file", root="../../../examples/serve/demo")  # Could also use "s3"

Get available data openers and writers. 
Data opener and writer identifiers use the format `{data_type}:{format_name}:{storage_type}`.

In [ ]:
data_store.get_data_opener_ids()

In [ ]:
list(data_store.get_data_ids())

### Open COG

In [ ]:
ml_dataset = data_store.get_open("cog-example.tif")

In [ ]:
ml_dataset.num_levels

Inspect the new multi-level dataset. Check spatial chunking or `tile_size`:

In [ ]:
ml_dataset.grid_mapping

In [ ]:
for level in range(ml_dataset.num_levels):
    dataset_i = ml_dataset.get_dataset(level)
    display(dataset_i)